# ДОМАШНЕЕ ЗАДАНИЕ 3. Классификация текстовых документов
## Цель работы

Приобрести опыт решения практических задач по машинному обучению, таких как анализ и визуализация исходных данных, обучение, выбор и оценка качества моделей предсказания, посредством языка программирования Python.

## Библиотеки

In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import balanced_accuracy_score, recall_score, precision_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
# Логистическая регрессия, Наивный Байес: модель Бернулли, мультиномиальная модель
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
# Импортим библиотеку чтобы засекать время
import time


RANDOM_STATE = 123

## Вариант

In [117]:
surname = "Панфилкин" # Ваша фамилия

alp = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
w = [4, 42, 21, 21, 34,  1, 44, 26, 18, 43, 38, 26, 18, 43,  3, 49, 45,
        7, 42, 25,  4,  9, 36, 33, 31, 29,  5, 31,  4, 19, 24, 27, 33]
d = dict(zip(alp, w))
variant =  sum([d[el] for el in surname.lower()]) % 3 + 1
print("Ваш вариант - ", variant)

Ваш вариант -  3


In [118]:
# Загрузка данных из файла data/reviews.tsv (первый столбец - метки классов, второй - тексты)
data = pd.read_csv('data/reviews.tsv', sep='\t', names=['target', 'text'])
# Вытащим X и y из датафрейма
X, y = data['text'], data['target']
# Отобразим датафрейм с данными
data

,target,text
0,0,unless bob crane is someone of particular inte...
1,1,"finds a way to tell a simple story , perhaps t..."
2,0,"ill-considered , unholy hokum ."
3,0,"nijinsky says , 'i know how to suffer' and if ..."
4,1,the auteur's ear for the way fears and slights...
...,...,...
10657,0,"it's mildly sentimental , unabashedly consumer..."
10658,0,so verbally flatfooted and so emotionally pred...
10659,0,alternative medicine obviously has its merits ...
10660,0,a by-the-numbers patient/doctor pic that cover...


In [124]:
# Разделим данные на обучающую и тестовую выборки в соотношении 80/20 с random_state=RANDOM_STATE
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_STATE)

# Создадим объект TfidfVectorizer с n-gram=1, слова в нижний регистр
vectorizer = TfidfVectorizer(ngram_range=(1, 1), lowercase=True)

# Обучим векторайзер на обучающей выборке
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [125]:
def get_metrics(model, X_train, y_train, X_test, y_test):
    """
    Функция для расчета метрик в виде словоря
    """
    start = time.time()
    model.fit(X_train, y_train)
    fit_time = time.time() - start
    start = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start

    return {
        'balanced_accuracy': balanced_accuracy_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'fit_time': fit_time,
        'predict_time': predict_time
    }

In [126]:
models = {
    "K-ближайших соседей (n=5)": KNeighborsClassifier(n_neighbors=5),
    "Логистическая регрессия (С=1)": LogisticRegression(
        penalty='l2',
        fit_intercept=True,
        max_iter=100,
        C=1,
        solver='lbfgs',
        random_state=RANDOM_STATE
    ),
    # TODO Использовать бинарный вектор!
    "Наивный Байес: модель Бернулли (a=1)": BernoulliNB(alpha=1),
    "Наивный Байес: полиномиальная модель (a=1)": MultinomialNB(alpha=1)
}

# Определим качество моделей как датафрейм
metrics_dict = {}

for name, model in models.items():
    metrics = get_metrics(model, X_train_vectorized, y_train, X_test_vectorized, y_test)
    metrics_dict[name] = metrics
metrics_df = pd.DataFrame(metrics_dict).T
metrics_df

,balanced_accuracy,recall,precision,f1,fit_time,predict_time
K-ближайших соседей (n=5),0.724304,0.744382,0.716216,0.730028,0.002985,1.132183
Логистическая регрессия (С=1),0.770246,0.792135,0.759425,0.775435,0.107547,0.000000
Наивный Байес: модель Бернулли (a=1),0.782477,0.774345,0.787619,0.780925,0.005019,0.001000
Наивный Байес: полиномиальная модель (a=1),0.787158,0.784644,0.789077,0.786854,0.003001,0.001000
